In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

%pylab inline

import wget
from ufal.udpipe import Model, Pipeline
import gensim

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Populating the interactive namespace from numpy and matplotlib


In [2]:
#data = pd.read_csv("./Projects/tinkoff_ml/data/wiki-wiki/train.csv", sep='\t')
data = pd.read_csv("train.csv", sep='\t')
data.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,NaN,замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,NaN,"шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,NaN,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,NaN,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,NaN,великий князь литовский гедимин после успешной...


In [3]:
data.word.unique()

array(['замок', 'лук', 'суда', 'бор'], dtype=object)

In [4]:
data.loc[:,['word', 'gold_sense_id', 'context']]\
.groupby(['word']).count()['context'].sort_values(ascending=False)

word
замок    138
суда     135
лук      110
бор       56
Name: context, dtype: int64

In [5]:
data.loc[:,['word', 'gold_sense_id', 'context']]\
.groupby(['word', 'gold_sense_id'])['context'].count()

word   gold_sense_id
бор    1                 14
       2                 42
замок  1                100
       2                 38
лук    1                 65
       2                 45
суда   1                100
       2                 35
Name: context, dtype: int64

# 2

Используем модель `udpipe_syntagrus` которая будет присваивать части речи словам

In [7]:
# udpipe_url = 'http://rusvectores.org/static/models/udpipe_syntagrus.model'
# modelfile = wget.download(udpipe_url)

In [8]:
modelfile='udpipe_syntagrus.model'
model = Model.load(modelfile)
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
def tag_ud(text='Текст нужно передать функции в виде строки!'):
    processed = pipeline.process(text) # обрабатываем текст, получаем результат в формате conllu
    output = [l for l in processed.split('\n') if not l.startswith('#')] # пропускаем строки со служебной информацией
    tagged = [w.split('\t')[2].lower() + '_' + w.split('\t')[3] for w in output if w] # извлекаем из обработанного текста лемму и тэг
    tagged_propn = []
    propn  = []
    for t in tagged:
        if t.endswith('PROPN'):
            if propn:
                propn.append(t)
            else:
                propn = [t]
        else:
            if len(propn) > 1:
                name = '::'.join([x.split('_')[0] for x in propn]) + '_PROPN'
                tagged_propn.append(name)
            elif len(propn) == 1:
                tagged_propn.append(propn[0])
            tagged_propn.append(t)
            propn = []
    return tagged_propn

In [9]:
%%time
data['context'] = data['context'].apply(tag_ud)

CPU times: user 14.2 s, sys: 29.5 ms, total: 14.3 s
Wall time: 14.3 s


Удалим ненужные столбцы

In [10]:
data = data.drop(columns=['predict_sense_id', 'positions'])

In [11]:
data.head()

,context_id,word,gold_sense_id,context
0,1,замок,1,"[замок_NOUN, владимир_NOUN, мономах_NOUN, в_AD..."
1,2,замок,1,"[шильонский_ADJ, замок_NOUN, замокнуть_ADJ, ши..."
2,3,замок,1,"[проведение_NOUN, архитектурно_ADJ, -_PUNCT, а..."
3,4,замок,1,"[топить_NOUN, с_ADP, ._PUNCT, ,_PUNCT, литр_NO..."
4,5,замок,1,"[великий_ADJ, князь_NOUN, литовский_ADJ, гедим..."


Используем `taiga_upos_skipgram_300_2_2018` векторные эмбеддинги 

In [12]:
%%time
# model_url = 'http://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz'
# modelfile = wget.download(model_url)
m = 'taiga_upos_skipgram_300_2_2018.vec.gz'
if m.endswith('.vec.gz'):
    model_vec = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model_vec = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model_vec = gensim.models.Word2Vec.load(m)

CPU times: user 1min 16s, sys: 591 ms, total: 1min 16s
Wall time: 1min 16s


Оптимизируем

In [13]:
model_vec.init_sims(replace=True)

Представим каждый контекст в виде эмбеддинга, для этого переведем каждое слово в векторное прдеставление, сложим и усредним. Если слова нет в словаре, то мы его просто не учитываем 

In [14]:
def context2vec(arrayOfWords):
    sentenceVector = np.zeros(300)
    sent_len = len(arrayOfWords)
    for w in arrayOfWords:
        try:
            wordVec = model_vec.word_vec(w)
            sentenceVector = np.add(sentenceVector, wordVec)
        except KeyError:
            sent_len -= 1
    sentenceVector /= sent_len
    return sentenceVector

In [15]:
n_data = data.loc[data['word'] == 'лук'].copy()
n_data['context'] = n_data['context'].apply(context2vec)
n_data

,context_id,word,gold_sense_id,context
138,139,лук,1,"[-0.014157257648944125, 0.03565143039399434, 0..."
139,140,лук,1,"[0.016724002299209437, 0.026864033761861115, -..."
140,141,лук,1,"[0.014039089417328005, 0.025787442229161767, 0..."
141,142,лук,1,"[0.010698547778541552, 0.03715446975205422, 0...."
142,143,лук,1,"[0.0042534099547101144, 0.02873408781563151, -..."
143,144,лук,1,"[0.012119156075641513, 0.03438682829229427, -0..."
144,145,лук,1,"[0.008149869014195413, 0.019619280524665907, 0..."
145,146,лук,1,"[-0.02165675675496459, 0.01814380068646963, 0...."
146,147,лук,1,"[-0.010786637258918388, 0.03113669203594327, 0..."
147,148,лук,1,"[0.007892013303935528, 0.02649710145808058, 0...."


Используем KMeans для кластрезиции, признаки – эмбеддинги предложений, посчитаем ARI 

In [16]:
km = KMeans(n_clusters=2)
y_pred = km.fit_predict(n_data['context'].tolist())

In [17]:
ari = adjusted_rand_score(n_data['gold_sense_id'], y_pred)
ari

0.9278721470140009

Повторим тот же алгоритм для других слов

In [18]:
def calcARIforWord(word):
    n_data = data.loc[data['word'] == word].copy()
    n_data['context'] = n_data['context'].apply(context2vec)

    km = KMeans(n_clusters=2)
    y_pred = km.fit_predict(n_data['context'].tolist())
    
    ari = adjusted_rand_score(n_data['gold_sense_id'], y_pred)
    print(ari)
    return ari

In [19]:
a1 = calcARIforWord('замок')
a2 = calcARIforWord('суда')
a3 = calcARIforWord('лук')
a4 = calcARIforWord('бор')

0.19633906439972718
0.33798134638470767
0.9278721470140009
1.0


Среднее ARI

In [20]:
avg = (a1+a2+a3+a4)/4
avg

0.615548139449609